In [50]:
import torch
from tqdm.notebook import tqdm
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification, BertConfig, BertModel
import pandas as pd
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.utils import class_weight
import numpy as np
from torch.optim import Adam
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
import json
import random

In [51]:
pd.options.display.max_colwidth = 3000
pd.options.display.max_rows = 3000

In [52]:
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device = torch.device('cuda')
device

device(type='cuda')

In [53]:
#data = pd.read_csv('./Backup master data/master_data_6.csv', usecols=['ADR', 'text', 'category', 'type', 'date'])
# old_data = pd.read_csv('./oldnew.csv')
# data = pd.read_csv('./master_data_new.csv')
data=pd.read_csv('./master_data_V3_0.csv')
# print(old_data.shape)
print(data.shape)


(314492, 9)


/tmp/ipykernel_23012/3006793153.py:4: DtypeWarning: Columns (6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('./master_data_V3_0.csv')


In [54]:
# old_data.columns

In [55]:
data.columns

Index(['ADR', 'text', 'category', 'type', 'date', 'ogr/aug', 'page#', 'angle',
       'renamed category'],
      dtype='object')

In [56]:
# old_data['type'].value_counts()

In [57]:
data['type'].value_counts()

LD      280866
NMIC     33626
Name: type, dtype: int64

In [58]:
# old_data_cat=old_data['category'].unique()

In [59]:
# new_data_cat=data['category'].unique()

In [60]:
# not_in_newdata=set()
# for cat in old_data_cat:
#     if cat not in new_data_cat:
#         not_in_newdata.add(cat)

In [61]:
# old_data_cat

In [62]:
# new_data_cat

In [63]:
# old_data[(old_data['category']=='NotClassified')].shape

In [64]:
# for dd in not_in_newdata:
#     print(dd)
# #     print(old_data[old_data['category']==dd]['org/aug'].value_counts())
#     tdf=pd.DataFrame(old_data[(old_data['category']==dd) & (old_data['org/aug']=='org')])
#     print(tdf.shape[0])
#     tdf['page#']=[-1]*tdf.shape[0]
#     tdf['angle']=[0]*tdf.shape[0]
# #     break
#     data=data.append(tdf)

In [65]:
data.isnull().sum()

ADR                     0
text                 1649
category                0
type                    0
date                    0
ogr/aug                 0
page#                   0
angle                  17
renamed category    84347
dtype: int64

In [66]:
data.dropna(inplace=True)
data.shape

(229177, 9)

In [67]:
data['category'].value_counts()

Tax Return - Personal                                 98667
Bank Statement                                        31728
Tax Return - Business                                 20913
Retirement Account Statement(s)                        9797
Paystub                                                8463
Hazard Insurance Dec Page - Initial                    8187
W-2(s) / 1099(s)                                       6617
Condominium Documents                                  6348
Rental Agreements(s)                                   5778
Mortgage Statement                                     5735
Purchase Agreement                                     4763
Divorce Decree / Child Support                         3619
Divorce Decree                                         3220
Title Report                                           3175
Escrow/Closing Instructions                            1995
K1s                                                    1389
Driver's License                        

In [68]:
n = data[data['category']=='Tax Return - Personal'].index.tolist()
data.drop(index= random.sample(n, 60000), inplace=True)

In [69]:
data['category'].value_counts()

Tax Return - Personal                                 38667
Bank Statement                                        31728
Tax Return - Business                                 20913
Retirement Account Statement(s)                        9797
Paystub                                                8463
Hazard Insurance Dec Page - Initial                    8187
W-2(s) / 1099(s)                                       6617
Condominium Documents                                  6348
Rental Agreements(s)                                   5778
Mortgage Statement                                     5735
Purchase Agreement                                     4763
Divorce Decree / Child Support                         3619
Divorce Decree                                         3220
Title Report                                           3175
Escrow/Closing Instructions                            1995
K1s                                                    1389
Driver's License                        

In [70]:
temp = []
for category in data['category']:
    if category == 'Divorce Decree':
        temp.append('Divorce Decree / Child Support')
    elif category == 'Escrow / Closing Instructions':
        temp.append('Escrow/Closing Instructions')
    else:
#         temp.append('focused')
        temp.append(category)
data['category'] = temp

In [71]:
# data['org/aug'].unique()

### Training

In [72]:
data['category'].unique()

array(['Setup Submission Form', 'VA - DD214 Member Copy for Discharge',
       'Bank Statement', 'Retirement Account Statement(s)', 'SSN Card',
       'Divorce Decree / Child Support', 'Social Security Award Letter',
       'Award Letter(s)', 'Tax Return - Business', 'Flood Certificate',
       'Paystub', 'Hazard Insurance Dec Page - Initial', 'K1s',
       'Tax Return - Personal', 'Mortgage Statement', "Driver's License",
       'Permanent Resident Card', 'W-2(s) / 1099(s)',
       'Purchase Agreement', 'Rental Agreements(s)',
       'Escrow/Closing Instructions', 'Trust Certification',
       'Bankruptcy Papers', 'Condominium Documents',
       'NOTE with all Attachments', 'Title Report',
       'Letter of Explanation', 'Non-Mortgage Account Statement',
       'Lender Approval', 'Disclosures', 'Credit Report', '4506-C',
       'Title - Other', 'IRS Transcripts', 'Income - Other',
       'Property - Other', 'W-9', 'Others', '4506-T', '1003',
       'Verification of business (VOB)/SOS 

In [73]:
data[data['category']=='Non-Mortgage Account Statement'].head(5)

,ADR,text,category,type,date,ogr/aug,page#,angle,renamed category
111629,ADR-2022-200000370062,"AMERICAN Platinum Card® p. 1/12 EXPRESS JAMES C RAWSON Customer Care: 1-800-525-3355 ITY: Closing Date 04/13/22 Use Relay 711 Account Ending 5-82006 Website: americanexpress.com Membership Rewards® Points New Balance $5,365.93 Available and Pending as of 03/31/22 Minimum Payment Due $2,721.72 688 For up to date point balance and full program details, visit membershiprewards.com Payment Due Date 05/08/22 Account Summary Pay In Full Late Payment Warning: If we do not receive your Minimum Payment Due by the Payment Due Date of 05/08/22, you may have to pay a late fee of up to Previous Balance $1,686.37 $40.00 and your Pay Over Time APR may be increased to the Penalty APR of Payments/Credits $1,741.82 29.99%. New Charges +$2,723.21 Fees +$0.00 New Balance $2,667.76 Pay Over Time and/or Cash Advance Previous Balance $5,441.91 Minimum Payment Warning: If you have a Pay Over Time and/or Cash Advance Payments/Credits -$12,841.21 balance and you make only the minimum payment each period, you will pay more in New Pay Over Time Charges +$9,203.85 interest and it will take you longer to pay off your balance. For example: New Cash Advances +$0.00 If you make no additional You will pay off the balance And you will pay an Fees +$870.00 charges and each month shown on this statement in estimated total of ... Interest Charged +$23.62 New Balance $2,698.17 you pay .. about ... Minimum Due $53.96 Only the Account Total 10 years $5,478 Minimum Payment Due Previous Balance $7,128.28 Payments/Credits -$14,583.03 If you would like information about credit counseling services, call 1-888-733-4139. New Charges +$11,927.06 See page 2 for important information about your account. New Cash Advances +$0.00 Fees +$870.00 Please refer to the IMPORTANT NOTICES section on Interest Charged +$23.62 pages 11 - 12. New Balance $5,365.93 Minimum Payment Due $2,721.72 We will debit your bank account for your payment of $2,721.72 on 04/28/22. This date may not be the same date your bank will debit your Pay Over Time Limit $22,500.00 bank account. Any inquiry to American Express concerning this debit Available Pay Over Time Limit $19,801.83 should be made before 04/26/22. If your AutoPay payment is less than your Minimum Payment Due, we must receive an additional payment for at least the difference by 05/08/22. Continued on page 3 Payment Coupon Pay by Computer Pay by Phone Account Ending 5-82006 Do not staple or use paper clips americanexpress.com/pbc 1-800-472-9297 Enter 15 digit account # on all payments. Make check payable to American Express. JAMES C RAWSON Payment Due Date 30 SANTA ANA LOOP 05/08/22 PLACITAS NM 87043-9440 New Balance $5,365.93 AutoPay Amount $2,721.72 See reverse side for instructions AMERICAN EXPRESS S P.O. BOX 650448 on how to update your address, DALLAS TX 75265-0448 Amount Enclosed phone number, or email. 0000349992556891747 000536593000272172 10",Non-Mortgage Account Statement,NMIC,20220722194403,org,509,0.0,No
111630,ADR-2022-200000370062,"JAMES C RAWSON Account Ending 5-82006 p. 2/12 Payments: Your payment must be sent to the payment address shown on effect on the date of your charge. Charges converted by establishments will your statement and must be received by 5 p.m. local time at that address to be billed at the rates such establishments use. be credited as of the day it is received. Payments we receive after 5 p.m. will Credit Balance: A credit balance (designated CR) shown on this statement not be credited to your Account until the next day. Payments must also: (1) represents money owed to you. If within the six-month period following include the remittance coupon from your statement; (2) be made with a the date of the first statement indicating the credit balance you do not single check drawn on a US bank and payable in US dollars, or with a request a refund or charge enough to use up the credit balance, we will negotiable instrument pa

In [74]:
temp = []

cats = ['Retirement Account Statement(s)',
        'Mortgage Statement',
        'Non-Mortgage Account Statement',
        'Bank Statement',
        'Purchase Agreement',
        'Rental Agreements(s)',
        'Hazard Insurance Dec Page - Initial',
        'Divorce Decree / Child Support',
        'Bankruptcy Papers']

# cats = ['Retirement Account Statement(s)',
#         'Mortgage Statement',
#         'Bank Statement']
        
        
for category in data['category']:
    if category not in cats:
        temp.append('others')
    else:
#         temp.append('focused')
        temp.append(category)
data['new_category'] = temp

In [75]:
'''label_dict = {}
label_dict['focused'] = 1
label_dict['others'] = 0
label_dict'''

"label_dict = {}\nlabel_dict['focused'] = 1\nlabel_dict['others'] = 0\nlabel_dict"

In [76]:
possible_labels = data.new_category.unique()

label_dict = {}
label_dict['others']=0
for i,cat in enumerate(cats):
    label_dict[cat]=i+1
label_dict

{'others': 0,
 'Retirement Account Statement(s)': 1,
 'Mortgage Statement': 2,
 'Non-Mortgage Account Statement': 3,
 'Bank Statement': 4,
 'Purchase Agreement': 5,
 'Rental Agreements(s)': 6,
 'Hazard Insurance Dec Page - Initial': 7,
 'Divorce Decree / Child Support': 8,
 'Bankruptcy Papers': 9}

In [77]:
data['label'] = data.new_category.replace(label_dict)
data.head(1)

,ADR,text,category,type,date,ogr/aug,page#,angle,renamed category,new_category,label
0,ADR-2022-100004473184,"loanDepot Branch: 102 - Cerritos Branch - Cerritos, CA NMLS # 174457 Set-Up Submission Form Borrower: Suzi Builder Loan Officer: POS Banker Credit Reference: 116287576930000 DU Case file ID: 1626158786 C.O.E.: 8/7/2022 Submitted: 7/7/2022 Prospect Loan#: 3001618980 Sales Price: $500,500.00 Appraised Value: $500,500.00 Loan Amt: $250,000.00 Rate: 5.250% Origination: $0.00 Discount: 0.086% LTV: 49.950% CLTV:49.950% Property Address: 432 Lake Front, Orwigsburg, PA 17961 Community Name: Loan Purpose: X Purchase Rate/Term Refi If Applicable 2nd loan Rate Term: 180 Cash-Out Refi Streamline Refi VA Non-Allowable Loan Type: X Conventional FHA USDA Seller Paid VA Other {Funding Fee: $ Exempt} Lender Paid SPECIFY PRODUCT CODE MSA/Builder FROM RATE SHEET Must select none or appropriate (Code is located at the bottom of the column of the program you are pricing) MSA/Desk Rental/Builder Bond: Yes Program 1: Program 2: Construction: Property Type: New Construction X SFR Detached SFR Attached Low Rise Condo Mid Rise Condo High Rise Condo Manufactured Home PUD Modular 2 Unit 3 Unit 4 Unit Co-op Other Occupancy Status Escrows Seller Credit: $ 0.00 X Primary 2nd Home Investment Impound: Transfer Taxes: $ 0.00 Waive Hazard: $ 100.00 Taxes: $ 0.00 MI: $ 0.00 Flood: $ Title Company Title Company Name: ServiceLink, LLC Contact: Address: 1355 Cherrington Pkwy, Moon Township, PA Phone: (800)439-5451 Fax: Email: kim.arndt@svclnk.com Hazard Insurance Insurance Company: Contact: Address: Phone: Fax: Email: Real Estate Agents: Buyer's Agent: Phone: Fax: Email: Seller's Agent: Phone: Fax: Email: Loan Officers: Please use this section to make comments regarding the loan. Mortgage Insurance - MI Lock Status Lender Paid MI (LPMI) Split MI Option Locked: Yes X No #Days 45 Borrower Paid MI (BPMI) Program 1 .05 Program 2 .75 Monthly Program 3 1.00 Single Premium Program 4 1.25 Single Premium Financed Program 5 1.50 Single Premium - Split cash/financed Split Premium (UFMIP cash) Program 6 1.75 100000 Split Premium (UFMIP financed) Empower Form (05/2014)",Setup Submission Form,LD,20220722123724,org,1,0.0,No,others,0


In [80]:
data = data.sample(frac=0.50, random_state=10)

In [82]:
data.shape

(84588, 11)

In [83]:
df = data.sample(frac=0.80, random_state=10)
df.shape

(67670, 11)

In [84]:
test = data.drop(df.index).reset_index(drop=True)
test.shape

(16918, 11)

In [85]:
test['new_category'].value_counts()

others                                 9600
Bank Statement                         3146
Retirement Account Statement(s)         961
Hazard Insurance Dec Page - Initial     812
Divorce Decree / Child Support          718
Rental Agreements(s)                    573
Mortgage Statement                      569
Purchase Agreement                      417
Bankruptcy Papers                        99
Non-Mortgage Account Statement           23
Name: new_category, dtype: int64

In [86]:
df['new_category'].value_counts()

others                                 37940
Bank Statement                         12673
Retirement Account Statement(s)         3983
Hazard Insurance Dec Page - Initial     3245
Divorce Decree / Child Support          2699
Rental Agreements(s)                    2392
Mortgage Statement                      2279
Purchase Agreement                      1971
Bankruptcy Papers                        399
Non-Mortgage Account Statement            89
Name: new_category, dtype: int64

In [87]:
classweight = torch.tensor(class_weight.compute_class_weight(class_weight='balanced',
                                                             classes=df['new_category'].unique().tolist(),
                                                             y=df['new_category'].values.tolist()), dtype=torch.float)
classweight

tensor([ 0.1784,  1.6990,  0.5340,  3.4333,  2.9693,  2.0854,  2.8290,  2.5072,
        16.9599, 76.0337])

In [88]:
df.reset_index(drop=True, inplace=True)
df.head(1)

,ADR,text,category,type,date,ogr/aug,page#,angle,renamed category,new_category,label
0,ADR-2022-100004517751,"12/31/20 2020 FEDERAL DEPRECIATION SCHEDULE PAGE 2 J&D TRANSPORT INC 82-5297252 PRIOR CUR SPECIAL 179/ PRIOR SALVAG DATE DATE COST/ BUS. 179 DEPR. BONUS/ DEC. BAL /BASIS DEPR. PRIOR CURRENT NO. DESCRIPTION ACQUIRED SOLD BASIS PCT. BONUS ALLOW. SP. DEPR. DEPR. REDUCT BASIS DEPR. METHOD LIFE RATE DEPR. 21 WELDER ** 11/26/20 5,073 5,073 200DB HY 5 .20000 1,015 22 VAN TRAILER 53010 ** 9/24/20 3,000 3,000 200DB HY 5 .20000 600 23 VAN TRAILER 45871 ** 9/24/20 3,000 3,000 200DB HY 5 .20000 600 TOTAL MACHINERY AND EQUIPME 115,353 0 0 16,040 0 0 99,313 12,186 27,174 TOTAL DEPRECIATION 490,418 0 0 16,040 0 0 474,378 72,030 138,093 GRAND TOTAL DEPRECIATION 490,418 0 0 16,040 0 0 474,378 72,030 138,093 == DEPRECIATION ASSETS SOLD 3,040 0 0 3,040 0 0 0 0 0 DEPR REMAINING ASSETS 487,378 0 0 13,000 0 0 474,378 72,030 138,093 **ASSET INCLUDED IN UNADJUSTED BASIS IMMEDIATELY AFTER ACQUISITION FOR THE QBI CALCULATION.",Tax Return - Business,LD,20220722123724,org,34,0.0,No,others,0


In [89]:
'''possible_labels = df.new_category.unique()

label_dict = {}
label_dict['others']=0
for i,cat in enumerate(cats):
    label_dict[cat]=i+1
#for index, possible_label in enumerate(possible_labels):
#    label_dict[possible_label] = index
label_dict'''

"possible_labels = df.new_category.unique()\n\nlabel_dict = {}\nlabel_dict['others']=0\nfor i,cat in enumerate(cats):\n    label_dict[cat]=i+1\n#for index, possible_label in enumerate(possible_labels):\n#    label_dict[possible_label] = index\nlabel_dict"

In [90]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.20, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

In [91]:
df['data_type'] = ['not_set']*df.shape[0]
df.head(2)

,ADR,text,category,type,date,ogr/aug,page#,angle,renamed category,new_category,label,data_type
0,ADR-2022-100004517751,"12/31/20 2020 FEDERAL DEPRECIATION SCHEDULE PAGE 2 J&D TRANSPORT INC 82-5297252 PRIOR CUR SPECIAL 179/ PRIOR SALVAG DATE DATE COST/ BUS. 179 DEPR. BONUS/ DEC. BAL /BASIS DEPR. PRIOR CURRENT NO. DESCRIPTION ACQUIRED SOLD BASIS PCT. BONUS ALLOW. SP. DEPR. DEPR. REDUCT BASIS DEPR. METHOD LIFE RATE DEPR. 21 WELDER ** 11/26/20 5,073 5,073 200DB HY 5 .20000 1,015 22 VAN TRAILER 53010 ** 9/24/20 3,000 3,000 200DB HY 5 .20000 600 23 VAN TRAILER 45871 ** 9/24/20 3,000 3,000 200DB HY 5 .20000 600 TOTAL MACHINERY AND EQUIPME 115,353 0 0 16,040 0 0 99,313 12,186 27,174 TOTAL DEPRECIATION 490,418 0 0 16,040 0 0 474,378 72,030 138,093 GRAND TOTAL DEPRECIATION 490,418 0 0 16,040 0 0 474,378 72,030 138,093 == DEPRECIATION ASSETS SOLD 3,040 0 0 3,040 0 0 0 0 0 DEPR REMAINING ASSETS 487,378 0 0 13,000 0 0 474,378 72,030 138,093 **ASSET INCLUDED IN UNADJUSTED BASIS IMMEDIATELY AFTER ACQUISITION FOR THE QBI CALCULATION.",Tax Return - Business,LD,20220722123724,org,34,0.0,No,others,0,not_set
1,ADR-2022-100004484554,"Morgan Stanley CLIENT STATEMENT | For the Period April 1-30, 2022 Page 23 of 62 Select UMA Active Assets Account MSL FBO FRANK PELLI & Account Summary 409-077646-303 ANITA K PELLI JT TEN ADDITIONAL ACCOUNT INFORMATION This Pericd This Year Category (4/1/22-4/30/22) (1/1/22-4/30/22) Foreign Tax Paid $107.89 $337.97 000042 M58820M1 001068",Retirement Account Statement(s),LD,20220722123724,org,23,0.0,No,Retirement Account Statement(s),1,not_set


In [92]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [93]:
df.groupby(['new_category', 'label', 'data_type']).count()

ADR   text  category  \
new_category                        label data_type                           
Bank Statement                      4     train      10138  10138     10138   
                                          val         2535   2535      2535   
Bankruptcy Papers                   9     train        319    319       319   
                                          val           80     80        80   
Divorce Decree / Child Support      8     train       2159   2159      2159   
                                          val          540    540       540   
Hazard Insurance Dec Page - Initial 7     train       2596   2596      2596   
                                          val          649    649       649   
Mortgage Statement                  2     train       1823   1823      1823   
                                          val          456    456       456   
Non-Mortgage Account Statement      3     train         71     71        71   
                                          val           18     18        18   
Purchase Agreement                  5     train       1577   1577      1577   
                                          val          394    394       394   
Rental Agreements(s)                6     train       1914   1914      1914   
                                          val          478    478       478   
Retirement Account Statement(s)     1     train       3187   3187      3187   
                                          val          796    796       796   
others                              0     train      30352  30352     30352   
                                          val         7588   7588      7588   

                                                      type   date  ogr/aug  \
new_category                        label data_type                          
Bank Statement                      4     train      10138  10138    10138   
                                          val         2535   2535     2535   
Bankruptcy Papers                   9     train        319    319      319   
                                          val           80     80       80   
Divorce Decree / Child Support      8     train       2159   2159     2159   
                                          val          540    540      540   
Hazard Insurance Dec Page - Initial 7     train       2596   2596     2596   
                                          val          649    649      649   
Mortgage Statement                  2     train       1823   1823     1823   
                                          val          456    456      456   
Non-Mortgage Account Statement      3     train         71     71       71   
                                          val           18     18       18   
Purchase Agreement                  5     train       1577   1577     1577   
                                          val          394    394      394   
Rental Agreements(s)                6     train       1914   1914     1914   
                                          val          478    478      478   
Retirement Account Statement(s)     1     train       3187   3187     3187   
                                          val          796    796      796   
others                              0     train      30352  30352    30352   
                                          val         7588   7588     7588   

                                                     page#  angle  \
new_category                        label data_type                 
Bank Statement                      4     train      10138  10138   
                                          val         2535   2535   
Bankruptcy Papers                   9     train        319    319   
                                          val           80     80   
Divorce Decree / Child Support      8     train       2159   2159   
                                          val          540    540   
Hazard Insurance Dec Page - Initial 7     train

In [94]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
import glob
import math

path = "./tempmortbert/model"

for modelpath in glob.iglob(path):
    print('Model: ', modelpath)
    tokenizer = BertTokenizer.from_pretrained(modelpath, use_fast = False, do_lower_case=True)
#     model = AutoModelForMaskedLM.from_pretrained(modelpath)


In [95]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    truncation=True,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512,
    truncation=True,
    return_tensors='pt'
)

/data/mmortgage/MultiPage_Classification/classification_service/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2285: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


KeyboardInterrupt: 

In [42]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

In [43]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [44]:
batch_size=32
dataloader_train = DataLoader(dataset_train, 
                              shuffle=True,
                              num_workers=0, 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   shuffle=False,
                                   num_workers=0, 
                                   batch_size=batch_size)

In [45]:
len(dataloader_train)

3384

In [46]:
len(dataloader_train)*32

108288

In [47]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, 
                 dropout=0.1, 
                 BERTconfig=False, 
                 attention_heads=12, 
                 hidden_layers=12, 
                 numclass=1,
                 hidden_dropout_prob=0.1,
                 hidden_act='gelu',
                 position_embedding_type='absolute'):
        
        #self.attention_heads = attention_heads
        super(BertClassifier, self).__init__()
        
        if BERTconfig:
            configuration = BertConfig(num_attention_heads= attention_heads, 
                                       num_hidden_layers= hidden_layers)   
        else:
            configuration = BertConfig()
        #self.bert = BertModel.from_pretrained('bert-base-uncased', output_attentions=attention)
        #print(configuration)
        self.bert = BertModel(configuration)
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, numclass, bias=True)

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)

        return linear_output

In [49]:
ATTENTION = 12
HIDDEN_LAYER = 1
CLASSES = len(label_dict)


model = BertClassifier(dropout=0.3, 
                       BERTconfig=True, 
                       attention_heads=ATTENTION,
                       hidden_layers=HIDDEN_LAYER,
                       numclass= CLASSES,
                       hidden_dropout_prob=0.3,
                       position_embedding_type='absolute')

model.resize_token_embeddings(len(tokenizer))
model.to(device)
model.train()

BertClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru

In [ ]:
EPOCH = 3

save_model = 250

criterion = nn.CrossEntropyLoss(weight=classweight.to(device), reduction='mean')

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8,
                  no_deprecation_warning=True)
                  

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=250,
                                            num_training_steps=len(dataloader_train)*EPOCH)


logs = []
i=0

for epoch in range(EPOCH):
    
    print('epoch--- ',epoch+1)
    
    for train_input in tqdm(dataloader_train):
        i=i+1
        model.zero_grad()
        input_id = train_input[0].to(device)
        mask = train_input[1].to(device)
        train_label = train_input[2].to(device)
        output = model(input_id, mask)
        batch_loss = criterion(output, train_label)
        batch_loss.backward()
        optimizer.step()
        scheduler.step()
        
        if(i%save_model==0):
            label_train =  torch.tensor([], dtype=torch.uint8).to(device)
            pred_train = torch.tensor([], dtype=torch.uint8).to(device)
            total_loss_train = 0

            for train_eval in dataloader_train:
                input_id = train_eval[0].to(device)
                mask = train_eval[1].to(device)
                train_label = train_eval[2].to(device)
                output = model(input_id, mask)
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                label_train = torch.cat((label_train, train_label), 0)
                pred_train = torch.cat((pred_train, output.argmax(dim=1)), 0)

            label_train = label_train.detach().cpu().numpy()
            pred_train = pred_train.detach().cpu().numpy()
            total_acc_train = accuracy_score(label_train, pred_train)
            total_f1_train = f1_score(label_train, pred_train, average='weighted')
            total_loss_train = total_loss_train/(len(dataloader_train)*batch_size)
            


            label_val =  torch.tensor([], dtype=torch.uint8).to(device)
            pred_val = torch.tensor([], dtype=torch.uint8).to(device)
            total_loss_val = 0

            for val_input in dataloader_validation:
                input_id = val_input[0].to(device)
                mask = val_input[1].to(device)
                val_label = val_input[2].to(device)
                output = model(input_id, mask)
                batch_loss = criterion(output, val_label)
                total_loss_val += batch_loss.item()
                label_val = torch.cat((label_val, val_label), 0)
                pred_val = torch.cat((pred_val, output.argmax(dim=1)), 0)

            label_val = label_val.detach().cpu().numpy()
            pred_val = pred_val.detach().cpu().numpy()
            total_acc_val = accuracy_score(label_val, pred_val)
            total_f1_val = f1_score(label_val, pred_val, average='weighted')
            total_loss_val = total_loss_val/(len(dataloader_validation)*batch_size)
            
            print('train_accuracy: ', total_acc_train, ', train_f1: ', total_f1_train, ', train_loss: ', total_loss_train)
            print('val_accuracy: ', total_acc_val, ',  val_f1: ', total_f1_val, ', val_loss: ', total_loss_val)

            torch.save(model.state_dict(), 
                       f'/mnt/custommodel/BASE_BERT_encoder{HIDDEN_LAYER}_attention{ATTENTION}_epoch{i}.model')

            logs.append({'epoch': i,
                     'train_accuracy': total_acc_train,
                     'train_f1': total_f1_train,
                     'train_loss': total_loss_train,
                     'val_accuracy': total_acc_val,
                     'val_f1': total_f1_val,
                     'val_loss': total_loss_val})

In [ ]:
log = pd.DataFrame(logs)
log['val_loss'] = log['val_loss'].round(3)
log['train_loss'] = log['train_loss'].round(3)
#log.index = log.epoch
#del log['epoch']
log

In [ ]:
import matplotlib.pyplot as plt

fig,ax = plt.subplots(figsize=(20,10))

ax.plot(log.epoch, log.train_f1, color="red")
ax.plot(log.epoch, log.val_f1, color="green")

# set x-axis label
ax.set_xlabel("iterations", fontsize = 14)

# set y-axis label
ax.set_ylabel("F1", fontsize=14)

ax.set_xticks(log.epoch.values)

ax2=ax.twinx()
ax2.plot(log.epoch, log.train_loss,color="red")
ax2.plot(log.epoch, log.val_loss,color="green")

ax2.set_ylabel("LOSS", fontsize=14)
plt.grid(color = 'green', linestyle = '--', linewidth = 0.5)
plt.show()

In [ ]:
plt.figure(figsize=(20,10))
#plt.plot(log.epoch, log.train_accuracy, label= 'train_accuracy')
plt.plot(log.epoch, log.train_f1, label= 'train_f1')
#plt.plot(log.epoch, log.val_accuracy, label= 'val_accuracy')
plt.plot(log.epoch, log.val_f1, label='val_f1')
plt.legend(loc="upper left")
plt.xticks(log.epoch, rotation=90)
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(log.epoch, log.train_loss, label='train_loss')
plt.plot(log.epoch, log.val_loss, label='val_loss')
plt.legend(loc="upper right")
plt.xticks(log.epoch, rotation=90)
plt.grid(True)
plt.show()

In [ ]:
'''EPOCH = 5

criterion = nn.CrossEntropyLoss()

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8,
                  no_deprecation_warning=True)
                  

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=500,
                                            num_training_steps=len(dataloader_train)*EPOCH)

logs = []

for epoch in range(EPOCH):
    
    print('epoch--- ',epoch+1)
    total_loss_train = 0
    label_train =  torch.tensor([], dtype=torch.uint8).to(device)
    pred_train = torch.tensor([], dtype=torch.uint8).to(device)

    for train_input in tqdm(dataloader_train):
        model.zero_grad()
        input_id = train_input[0].to(device)
        mask = train_input[1].to(device)
        train_label = train_input[2].to(device)
        output = model(input_id, mask)
        label_train = torch.cat((label_train, train_label), 0)
        batch_loss = criterion(output, train_label)
        total_loss_train += batch_loss.item()
        pred_train = torch.cat((pred_train, output.argmax(dim=1)), 0)
        batch_loss.backward()
        optimizer.step()
        scheduler.step()
        #break
    
    label_train = label_train.detach().cpu().numpy()
    pred_train = pred_train.detach().cpu().numpy()
    total_acc_train = accuracy_score(label_train, pred_train)
    total_f1_train = f1_score(label_train, pred_train, average='weighted')
    total_loss_train = total_loss_train/(len(dataloader_train)*batch_size)

    label_val =  torch.tensor([], dtype=torch.uint8).to(device)
    pred_val = torch.tensor([], dtype=torch.uint8).to(device)
    total_loss_val = 0

    with torch.no_grad():
        for val_input in tqdm(dataloader_validation):
            input_id = val_input[0].to(device)
            mask = val_input[1].to(device)
            val_label = val_input[2].to(device)
            output = model(input_id, mask)
            label_val = torch.cat((label_val, val_label), 0)
            batch_loss = criterion(output, val_label)
            total_loss_val += batch_loss.item()
            pred_val = torch.cat((pred_val, output.argmax(dim=1)), 0)
            #break
    
    label_val = label_val.detach().cpu().numpy()
    pred_val = pred_val.detach().cpu().numpy()
    total_acc_val = accuracy_score(label_val, pred_val)
    total_f1_val = f1_score(label_val, pred_val, average='weighted')
    total_loss_val = total_loss_val/(len(dataloader_validation)*batch_size)
    
    torch.save(model.state_dict(), f'./model/BERT_encoder{HIDDEN_LAYER}_attention{ATTENTION}_epoch{epoch+1}.model')
    
    print('train_accuracy: ', total_acc_train, ', train_f1: ', total_f1_train, ', train_loss: ', total_loss_train)
    print('val_accuracy: ', total_acc_val, ',  val_f1: ', total_f1_val, ', val_loss: ', total_loss_val)
    print('\n')
    
    logs.append({'epoch': epoch+1,
                 'train_accuracy': total_acc_train,
                 'train_f1': total_f1_train,
                 'train_loss': total_loss_train,
                 'val_accuracy': total_acc_val,
                 'val_f1': total_f1_val,
                 'val_loss': total_loss_val})'''

In [ ]:
label_dict

### Evaluation

In [ ]:
#test['label'] = test.new_category.replace(label_dict)
test.head(2)

In [ ]:
test['new_category'].value_counts()

In [ ]:
def BERTEncode(df, size, shuffle=False):
    encoded_data = tokenizer.batch_encode_plus(
        df.text.values, 
        add_special_tokens=True, 
        return_attention_mask=True, 
        padding='max_length', 
        max_length=512, 
        truncation=True,
        return_tensors='pt')
    input_ids = encoded_data['input_ids']
    attention_masks = encoded_data['attention_mask']
    labels = torch.tensor(df.label.values)
    tensordataset = TensorDataset(input_ids, attention_masks, labels)
    dataloader = DataLoader(tensordataset,
                            shuffle=shuffle,
                            num_workers=0,
                            batch_size=size)
    return dataloader

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
dataloader_test = BERTEncode(test, 32)

In [ ]:
ATTENTION = 12
HIDDEN_LAYER = 1
CLASSES = len(label_dict)

model = BertClassifier(dropout=0.2, 
                       BERTconfig=True, 
                       attention_heads=ATTENTION,
                       hidden_layers=HIDDEN_LAYER,
                       numclass= CLASSES,
                       hidden_dropout_prob=0.1,
                       position_embedding_type='absolute')
#model = BertClassifier(BERTconfig=False, numclass= CLASSES)

model.to(device)

model.load_state_dict(torch.load('/mnt/custommodel/BASE_BERT_encoder1_attention12_epoch7250.model', map_location=torch.device('cuda')))
model.eval()

In [ ]:
def validate(dataloader, model):
    actual =  torch.tensor([], dtype=torch.uint8).to(device)
    predicted = torch.tensor([], dtype=torch.uint8).to(device)
    cr = []

    for input in tqdm(dataloader):
        input_id = input[0].to(device)
        mask = input[1].to(device)
        labels = input[2].to(device)
        actual = torch.cat((actual, labels), 0)
        with torch.no_grad():
            output = model(input_id, mask)
        predicted = torch.cat((predicted, output.argmax(dim=1)), 0)
        for logits in output:
            score = torch.nn.functional.softmax(logits, dim=0)
            cr.append(np.float64(score.max(dim=0)[0].detach().cpu().numpy()))
    
    actual = actual.detach().cpu().numpy()
    predicted = predicted.detach().cpu().numpy()
    
    total_acc_test = accuracy_score(actual, predicted)
    total_f1_test = f1_score(actual, predicted, average='weighted')
    print('Test Accuracy:', round(total_acc_test,2))
    print('Test f1:', round(total_f1_test,2))
    return (actual, predicted, cr)

In [ ]:
actual, predicted, cr =validate(dataloader_test, model)

In [ ]:
label_dict_inv=dict()
for label in label_dict:
    label_dict_inv[label_dict[label]]=label
label_dict_inv 

In [ ]:
checkfor = range(len(label_dict))
#checkfor = [0]
final_predicted = []
    

for check in checkfor:
    total = 0
    correct = 0
    notclassified = 0
    score = []
    for a,p, c in zip(actual, predicted, cr):
             
        if a == check:
            total+=1
            if (p == check):
                correct+=1
            elif (p!=check):
                notclassified+=1
                score.append(c)
                
    if total!=0:
        acc = round(correct/total,2)
        print('accuracy for {}: {}/{}={}'.format(label_dict_inv[check],correct,total,acc))
        print('notclassified {}'.format(notclassified))
        if notclassified!=0:
            print('notclassified score for {}'.format(np.median(score)))
    print('\n')

In [ ]:
test[test['category']=='Mortgage Statement']['org/aug'].value_counts()

In [ ]:
test[test['category']=='Hazard Insurance Dec Page - Initial'].head(5)

In [ ]:
test_ocr=test[(test['category']=='Non-Mortgage Account Statement') &(test['ADR']=='ADR-2022-200000370062')]['text'].values[0]

In [ ]:
test_ocr=''.join([i for i in test_ocr if not i.isdigit()])

In [ ]:
test_ocr

In [ ]:
bert_tokens = tokenizer.tokenize(test_ocr)
cnt=0
for tup in zip(bert_tokens):
    print('{:<12}'.format(tup[0]))
    cnt+=1
    if cnt>=512:
        break

In [ ]:
def predict(model, encoded_data_text, labels, thresh):

    input_ids_text = encoded_data_text['input_ids'].to(device)
    attention_masks_text = encoded_data_text['attention_mask'].to(device)

    with  torch.no_grad(): 
        outputs = model(input_ids_text, attention_masks_text)
        
    scores = torch.nn.functional.softmax(outputs.detach().cpu(), dim=1).numpy()[0]
    
    if np.max(scores)>thresh:
        return (labels[np.argmax(scores)],np.max(scores))
    else:
        return ('others', np.max(scores))

In [ ]:
txts=[test_ocr]

In [ ]:
from concurrent.futures import ThreadPoolExecutor

pred_label = []
THRESHOLDS =0.9
# for txt in global_validate['text'].values:
for txt in txts:
#for txt in zip(global_validate['text'].values, global_validate['Ref. No.'].values, global_validate['text'].values:
    
    encoded_data = tokenizer.encode_plus(
            str(txt), 
            add_special_tokens=True, 
            return_attention_mask=True, 
            padding='max_length', 
            max_length=512, 
            truncation=True,
            return_tensors='pt')

    predictions=predict(model,encoded_data,label_dict_inv,THRESHOLDS)
#     with ThreadPoolExecutor() as executor:
#         predictions = executor.map(predict, BERT_MODELS, [encoded_data]*len(labels), labels, THRESHOLDS)
    predictions = list(predictions)
#     print(predictions)
#     if(predictions[0][0]=='focused'):
#         classifications = list(filter(lambda x: x[0]!='others', predictions[1:]))
#         if len(classifications)==0:
#             final_class = sorted(predictions[1:],key=lambda x: x[1], reverse=True)[0]
#         else:
#             final_class = sorted(classifications,key=lambda x: x[1], reverse=True)[0]

#         '''if final_class[1]>0.90:
#             final_class = final_class
#         else:
#             final_class = ('others',final_class[1])'''
#     else:
#         final_class = ('others',predictions[0][1])
#     pred_label.append(label[final_class[0]])
    print(predictions)
    print('\n')

### GLOBAL_VALIDATION

In [ ]:
global_validate = pd.read_csv('./global_validate.csv')
global_validate.head(2)

In [ ]:
#global_validate = global_validate[global_validate['new_text_format']=='new']

In [ ]:
global_validate['category'].value_counts()

In [ ]:
#global_validate.drop(global_validate[global_validate['category']=='Tax Return'].index, inplace=True)

In [ ]:
count=0
index=[]
for i,row in global_validate.iterrows():
    if row['ADR'] in data['ADR'].values.tolist():
        if row['text'] in data['text'][data['ADR']==row['ADR']].values.tolist():
            count+=1
            index.append(i)
count

In [ ]:
#del global_validate['new_category']
#global_validate.drop(index=index, inplace=True)
global_validate.shape

In [ ]:
#global_validate['category'].value_counts()

In [ ]:
global_validate.to_csv('./global_validate.csv', header=True, index=False)

In [ ]:
temp = []
cats = ['Tax Return - Personal', 
        'Tax Return - Business', 
        'K1s',
        'Retirement Account Statement(s)',
        'Mortgage Statement',
        'Bank Statement',
        'Purchase Agreement',
        'Rental Agreements(s)',
        'Hazard Insurance Dec Page - Initial',
        'Divorce Decree / Child Support',
        'Bankruptcy Papers']

# cats = ['Retirement Account Statement(s)', 
#         'Mortgage Statement',
#         'Bank Statement']

for category in global_validate['category']:
    if category not in cats:
        temp.append('others')
    else:
        temp.append('focused')
global_validate['new_category'] = temp
global_validate.head(2)

In [ ]:
global_validate['new_category'].value_counts()

In [ ]:
global_validate['label'] = global_validate.new_category.replace(label_dict)
global_validate.head(1)

In [ ]:
global_validate['text']= [str(text) for text in global_validate['text']]

In [ ]:
def BERTEncode(df, size, shuffle=False):
    encoded_data = tokenizer.batch_encode_plus(
        df['new_format_text'].values, 
        add_special_tokens=True, 
        return_attention_mask=True, 
        padding='max_length', 
        max_length=512, 
        truncation=True,
        return_tensors='pt')
    input_ids = encoded_data['input_ids']
    attention_masks = encoded_data['attention_mask']
    labels = torch.tensor(df.label.values)
    tensordataset = TensorDataset(input_ids, attention_masks, labels)
    dataloader = DataLoader(tensordataset,
                            shuffle=shuffle,
                            num_workers=0,
                            batch_size=size)
    return dataloader

In [ ]:
dataloader = BERTEncode(global_validate, 32, False)

In [ ]:
ATTENTION = 12
HIDDEN_LAYER = 1
CLASSES = len(label_dict)


model = BertClassifier(dropout=0.2, 
                       BERTconfig=True, 
                       attention_heads=ATTENTION,
                       hidden_layers=HIDDEN_LAYER,
                       numclass= CLASSES,
                       hidden_dropout_prob=0.1,
                       position_embedding_type='absolute')

model.to(device)
model.load_state_dict(torch.load('/mnt/model/BERT_encoder1_attention12_epoch3000.model', map_location=torch.device('cuda')))
model.eval()

In [ ]:
actual, predicted, cr = validate(dataloader, model)

In [ ]:
global_validate.shape

In [ ]:
label_dict_inv=dict()
for label in label_dict:
    label_dict_inv[label_dict[label]]=label
label_dict_inv 

In [ ]:
label_dict

In [ ]:
threshholds = range(0,92,2)
checkfor = range(len(label_dict))

accuracy=[]
notclassified = []
for thresh in threshholds:
    
    final_predicted=[]
    for a,p,c in zip(actual, predicted, cr):
        if c>(thresh/100):
            final_predicted.append(p)
        else:
            final_predicted.append(0)
    
    acc = 0
    incorrect = 0
    for check in checkfor:
        total = 0
        correct = 0
        for a,p,c in zip(actual, final_predicted, cr):
            if a == check:
                total+=1
                if (p == check):
                    correct+=1
                elif (p!=check):
                    incorrect+=1
        if(total!=0):
            acc = acc+round(correct/total,2)
    #accuracy.append(acc/len(checkfor))
    accuracy.append(acc/len(global_validate['new_category'].unique()))
    notclassified.append(incorrect)

In [ ]:
fig,ax = plt.subplots(figsize=(20,10))

ax.plot(threshholds,
        accuracy,
        color="red", 
        marker="o")
# set x-axis label
ax.set_xlabel("threshold", fontsize = 14)
# set y-axis label
ax.set_ylabel("accuracy",
              color="red",
              fontsize=14)

ax.set_xticks(list(threshholds))
ax2=ax.twinx()
ax2.plot(threshholds, notclassified,color="blue",marker="o")
ax2.set_ylabel("notclassified",color="blue",fontsize=14)
plt.grid(color = 'green', linestyle = '--', linewidth = 0.5)
plt.show()

In [ ]:
checkfor = range(len(label_dict))
#checkfor = [0]
final_predicted = []

for a,p, c in zip(actual, predicted, cr):
    if c>0.90:
        final_predicted.append(p)
    else:
        final_predicted.append(0)
    

for check in checkfor:
    total = 0
    correct = 0
    notclassified = 0
    score = []
    for a,p, c in zip(actual, final_predicted, cr):
             
        if a == check:
            total+=1
            if (p == check):
                correct+=1
            #elif ((p!=check)&(c<0.90)):
            #    correct+=1
            #elif ((p!=check)&(c>0.90)):
            #    notclassified+=1
            elif (p!=check):
                notclassified+=1
                score.append(c)
                
    if total!=0:
        acc = round(correct/total,2)
        print('accuracy for {}: {}/{}={}'.format(label_dict_inv[check],correct,total,acc))
        print('notclassified {}'.format(notclassified))
        if notclassified!=0:
            print('notclassified score for {}'.format(np.median(score)))
    print('\n')

In [ ]:
global_validate['predlabel'] = list(final_predicted)
global_validate['predicted'] = [label_dict_inv[i] for i in final_predicted]
global_validate['cr'] = list(cr)
#global_validate.head(2)

In [ ]:
global_validate['category'][(global_validate['new_category']=='others')&
                            (global_validate['predicted']!='others')].value_counts()

In [ ]:
global_validate['new_category'].value_counts()

In [ ]:
global_validate[(global_validate['label']==0)&(global_validate['predlabel']!=0)].shape

In [ ]:
global_validate['new_category'].value_counts()

In [ ]:
global_validate.shape

### after deployment validation

In [ ]:
#global_validate = pd.read_csv('./global_validate.csv')
global_validate = pd.read_csv("BERT Testing_28.07.2022.csv")
global_validate.shape

In [ ]:
global_validate.head(1)

In [ ]:
global_validate['Actual Classification'].value_counts()

In [ ]:
# count=0
# index=[]
# for i,adr in enumerate(global_validate['ADR']):
#     if adr in data['ADR'].values.tolist():
#         count+=1
#         index.append(i)
# global_validate.drop(index=index, inplace=True)
# global_validate.shape

In [ ]:
label = {
    'others': 0,
    'Retirement Account Statement(s)': 1, 
    'Mortgage Statement': 2, 
    'Bank Statement': 3, 
    'Purchase Agreement': 4, 
    'Rental Agreements(s)': 5,
    'Divorce Decree / Child Support': 6,
    'Hazard Insurance Dec Page - Initial': 7,
    'Bankruptcy Papers': 8,
    'Tax Return - Personal': 9, 
    'Tax Return - Business': 10, 
    'K1s':11
}

In [ ]:
label_inv=dict()
for lab in label:
    label_inv[label[lab]]=lab
label_inv 

In [ ]:
temp = []

cats = list(label.keys())

for category in global_validate['Actual Classification']:
    if category not in cats:
        temp.append('others')
    else:
        temp.append(category)
global_validate['new_category'] = temp

In [ ]:
global_validate['label'] = global_validate.new_category.replace(label)
global_validate.head(2)

In [ ]:
global_validate['new_category'].value_counts()

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, 
                 dropout=0.1, 
                 BERTconfig=False, 
                 attention_heads=12, 
                 hidden_layers=12, 
                 numclass=1,
                 hidden_dropout_prob=0.1,
                 hidden_act='gelu',
                 position_embedding_type='absolute'):
        
        #self.attention_heads = attention_heads
        super(BertClassifier, self).__init__()
        
        if BERTconfig:
            configuration = BertConfig(num_attention_heads= attention_heads, 
                                       num_hidden_layers= hidden_layers)   
        else:
            configuration = BertConfig()
        #self.bert = BertModel.from_pretrained('bert-base-uncased', output_attentions=attention)
        #print(configuration)
        self.bert = BertModel(configuration)
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, numclass, bias=True)

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)

        return linear_output

In [ ]:
import configparser
import ast

device = torch.device('cuda')

configs = configparser.ConfigParser()
configs.read('./config.ini')
labels = []
BERT_MODELS = []
THRESHOLDS=[]

for config in configs.sections():
    label_ = ast.literal_eval(configs[config].get('categoryMAP'))
    labels.append(label_)
    THRESHOLDS.append(float(configs[config].get('cr_thresh')))
    
    HIDDEN_LAYERS = int(configs[config].get('hidden_layers'))
    ATTENTIONS = int(configs[config].get('attentions'))
    CLASSES = len(label_)
    MODEL_FOLDER = configs[config].get('modelfolder')
    model = BertClassifier(BERTconfig=True, attention_heads=ATTENTIONS, hidden_layers=HIDDEN_LAYERS, numclass= CLASSES)
    model.to(device)
    model.load_state_dict(torch.load('./model/{}/BERT.model'.format(MODEL_FOLDER), map_location=torch.device('cuda')))
    model.eval()
    BERT_MODELS.append(model)

In [ ]:
def predict(model, encoded_data_text, labels, thresh):

    input_ids_text = encoded_data_text['input_ids'].to(device)
    attention_masks_text = encoded_data_text['attention_mask'].to(device)

    with  torch.no_grad(): 
        outputs = model(input_ids_text, attention_masks_text)
        
    scores = torch.nn.functional.softmax(outputs.detach().cpu(), dim=1).numpy()[0]
    
    if np.max(scores)>thresh:
        return (labels[np.argmax(scores)],np.max(scores))
    else:
        return ('others', np.max(scores))

In [ ]:
txts = ['J.P.Morgan BRANDON R LIPMAN IRA Statement Period: April 01 - April 29, 2022 JPMS LLC IRA (Acct # 988-32917) JPMS LLC CUST. J.P. Morgan Self-Directed Investing Activity CASH FLOW SUMMARY This Period Year-to-Date Description $937.43 $937.40 Opening Cash Balance 0.01 0.04 Income $0.01 $0.04 Total Credits $0.00 0.00 Total Debits Net Cash Activity $0.01 $0.04 CLOSING CASH BALANCE $937.44 $937.44 "Opening Cash Balance" and "Closing Cash Balance" include Sweep Funds. SWEEP PROGRAM ACTIVITY CHASE IRA DEPOSIT SWEEP,JPMORGAN CHASE BANK NA, Symbol: QDERQ Debit Amount Credit Amount Transaction Description Quantity Price Date 937.43 OPENING BALANCE (0.01) MONTHLY INTEREST 0.01 01 Apr 2022 REINVEST REINVESTED 937.44 1 CLOSING BALANCE CHASE IRA DEPOSIT SWEEP,JPMORGAN CHASE BANK NA, Symbol: QDERQ Credit Amount Description Quantity Price Debit Amount Date Transaction 0.01 INTEREST MONTHLY INTEREST 01 Apr 2022 03/01-03/31 ($0.01) SWEEP PROGRAM ACTIVITY $0.01 Sweep Program Dividend/Interest ($0.01) Total Sweep Program Activity A - Average Cost B-Adjusted for Amortization or Accretion D - Acquisition Date = Date of Death E- Adjusted for Option Exercise or Assignment K- Gifted Security LT - Long Term MT - Mixed Term N-Noncovered Provide - Please provide this information ST - Short Term T- Cost Basis provided by Third Party W- Adjusted for Wash Sale Closing Methods: MLMG - Maximum Loss, Minimum Gain LIFO- Last In, First Out FIFO - First In, First Out HC-High Cost LC-Low Cost LTHC-Long Term, High Cost PRO - Pro Rata VSP - Specific Match (the closing transaction was specifically matched to this lot) Page 8 of 14 Please read the important disclosures at the end of the statement. For questions, please contact us using the information provided on the front of this statement. IMPORTANT INFORMATION STATEMENT SUMMARY RETIREMENT BROKERAGE']

In [ ]:
from concurrent.futures import ThreadPoolExecutor

pred_label = []

# for txt in global_validate['text'].values:
for txt in txts:
#for txt in zip(global_validate['text'].values, global_validate['Ref. No.'].values, global_validate['text'].values:
    
    encoded_data = tokenizer.encode_plus(
            str(txt), 
            add_special_tokens=True, 
            return_attention_mask=True, 
            padding='max_length', 
            max_length=512, 
            truncation=True,
            return_tensors='pt')

    with ThreadPoolExecutor() as executor:
        predictions = executor.map(predict, BERT_MODELS, [encoded_data]*len(labels), labels, THRESHOLDS)
    predictions = list(predictions)
    print(predictions)
    if(predictions[0][0]=='focused'):
        classifications = list(filter(lambda x: x[0]!='others', predictions[1:]))
        if len(classifications)==0:
            final_class = sorted(predictions[1:],key=lambda x: x[1], reverse=True)[0]
        else:
            final_class = sorted(classifications,key=lambda x: x[1], reverse=True)[0]

        '''if final_class[1]>0.90:
            final_class = final_class
        else:
            final_class = ('others',final_class[1])'''
    else:
        final_class = ('others',predictions[0][1])
    pred_label.append(label[final_class[0]])
    print(final_class)
    print('\n')

In [ ]:
label

In [ ]:
acc = accuracy_score(global_validate['label'].values, np.array(pred_label))
f1 = f1_score(global_validate['label'].values, np.array(pred_label), average='weighted')
acc,f1

In [ ]:
label_inv

In [ ]:
range(len(label_inv))

In [ ]:
checkfor = range(len(label_inv))
for check in checkfor:
    total = 0
    correct = 0
    notclassified = 0
    score = []
    for a,p in zip(global_validate['label'].values, np.array(pred_label)):
             
        if a == check:
            total+=1
            if (p == check):
                correct+=1
            elif (p!=check):
                notclassified+=1
                
    if total!=0:
        acc = round(correct/total,2)
        print('accuracy for {}: {}/{}={}'.format(label_inv[check],correct,total,acc))
        #print('notclassified {}'.format(notclassified))
        print('\n')

In [ ]:
global_validate['pred_label'] = pred_label
global_validate.head(1)

In [ ]:
global_validate['pred_category'] = global_validate['pred_label'].replace(label_inv )
global_validate.head(1)

In [ ]:
global_validate['new_category'].values

In [ ]:
temp = []
for i,j in zip(global_validate['new_category'].values, global_validate['pred_category'].values):
    if i==j:
        temp.append('PASS')
    else:
        temp.append('FAIL')
global_validate['local_P/F']= temp

In [ ]:
global_validate.to_csv('./valid.csv', header=True, index=False)

In [ ]:
global_validate[(global_validate['label']==0)&
                            (global_validate['pred_label']!=0)].shape

In [ ]:
bank 
morgage 
Retirement

In [ ]:
Purchase
Rental

In [ ]:
tax return
k1
w2
paystub

In [ ]:
import nlpaug.augmenter.sentence as nas
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.char as nac

In [ ]:
augc= nac.OcrAug() 
'''aug = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', 
                                model_type='bert',
                                action="insert", 
                                aug_p=0.80)'''
                                
#augs = nas.ContextualWordEmbsForSentenceAug(model_path='gpt2', top_p=0.5, batch_size=4)
#randomSentAug = nas.random.RandomSentAug(mode='neighbor', action='swap') <---

In [ ]:
txt = [data['text'][1],data['text'][0]]
augmented_data = augc.augment(txt)
#augmented_data = randomWordAug.augment(txt)

In [ ]:
txt

In [ ]:
augmented_data